### Deliverable 3: Optimize the Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
# application_df.head()

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(["EIN","NAME"], axis=1)
# application_df.head()

In [3]:
# Determine the number of unique values in each column.
n = application_df.nunique(axis=0)
print("No.of.unique values in each column :\n",n)

No.of.unique values in each column :
 APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df.APPLICATION_TYPE.value_counts()
# app_counts

In [5]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
# class_counts

#### 1st attempt to optimize model
#### ---Change binning for APPLICATION_TYPE and CLASSIFICATION into 2 bins each

In [6]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_counts[app_counts < 20000].index)
# replace_application

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     7262
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 10000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
Other    16973
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
# application_cat

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
# encode_df.head()

In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)

application_df = application_df.drop(application_cat,1)
# application_df.head()

In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, 
                          activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                2640      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5th epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback], verbose=0)


Epoch 00005: saving model to checkpoints\weights.05.hdf5

Epoch 00010: saving model to checkpoints\weights.10.hdf5

Epoch 00015: saving model to checkpoints\weights.15.hdf5

Epoch 00020: saving model to checkpoints\weights.20.hdf5

Epoch 00025: saving model to checkpoints\weights.25.hdf5

Epoch 00030: saving model to checkpoints\weights.30.hdf5

Epoch 00035: saving model to checkpoints\weights.35.hdf5

Epoch 00040: saving model to checkpoints\weights.40.hdf5

Epoch 00045: saving model to checkpoints\weights.45.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5


#### Results from change 1

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5907 - accuracy: 0.7089
Loss: 0.5907118916511536, Accuracy: 0.7089212536811829


In [17]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

#### 2nd attempt to optimize model
#### ---Change binning for APPLICATION_TYPE and CLASSIFICATION into 2 bins each
#### ---Change activation function on hidden layers from relu to tanh

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, 
                          activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                2640      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5th epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback], verbose=0)


Epoch 00005: saving model to checkpoints\weights.05.hdf5

Epoch 00010: saving model to checkpoints\weights.10.hdf5

Epoch 00015: saving model to checkpoints\weights.15.hdf5

Epoch 00020: saving model to checkpoints\weights.20.hdf5

Epoch 00025: saving model to checkpoints\weights.25.hdf5

Epoch 00030: saving model to checkpoints\weights.30.hdf5

Epoch 00035: saving model to checkpoints\weights.35.hdf5

Epoch 00040: saving model to checkpoints\weights.40.hdf5

Epoch 00045: saving model to checkpoints\weights.45.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5


#### Results from change 2

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5894 - accuracy: 0.7083
Loss: 0.5894148349761963, Accuracy: 0.70833820104599


In [22]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

#### 3rd attempt to optimize model
#### ---Change binning for APPLICATION_TYPE and CLASSIFICATION into 2 bins each
#### ---Change activation function on hidden layers to tanh
#### ---Add third hidden layer
#### ---Lower number of nodes in first 2 layers
#### ---Change epochs to 100 in the training regimen

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, 
                          activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 6         
Total params: 345
Trainable params: 345
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [35]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5th epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback], verbose=0)


Epoch 00005: saving model to checkpoints\weights.05.hdf5

Epoch 00010: saving model to checkpoints\weights.10.hdf5

Epoch 00015: saving model to checkpoints\weights.15.hdf5

Epoch 00020: saving model to checkpoints\weights.20.hdf5

Epoch 00025: saving model to checkpoints\weights.25.hdf5

Epoch 00030: saving model to checkpoints\weights.30.hdf5

Epoch 00035: saving model to checkpoints\weights.35.hdf5

Epoch 00040: saving model to checkpoints\weights.40.hdf5

Epoch 00045: saving model to checkpoints\weights.45.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00055: saving model to checkpoints\weights.55.hdf5

Epoch 00060: saving model to checkpoints\weights.60.hdf5

Epoch 00065: saving model to checkpoints\weights.65.hdf5

Epoch 00070: saving model to checkpoints\weights.70.hdf5

Epoch 00075: saving model to checkpoints\weights.75.hdf5

Epoch 00080: saving model to checkpoints\weights.80.hdf5

Epoch 00085: saving model to checkpoints\weights.85.hdf5

Epoch 00090: 

#### Results from change 3

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5891 - accuracy: 0.7089
Loss: 0.5890666842460632, Accuracy: 0.7089212536811829


In [37]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")